In [1]:
import KeyFunctions as kf

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
#Import the postcode list from the raw data gdf
# gdf = kf.load_obj(root_path,"raw_gdf")
name = "StPauls" 
gdf = kf.load_obj(root_path,"raw_gdf_" + name)

#2m29s

In [ ]:
#gdf["Type"].unique().tolist()

In [ ]:
#Initialise our postcodes gdf as a list of postcodes and their points
Postcodes = gdf.loc[gdf["Type"]=="Postcodes",["Name", "geometry"]].copy()
Postcodes = Postcodes.rename(columns={"Name": "Postcode"})

Postcodes = Postcodes.set_index("Postcode")

In [ ]:
#List out the points we will want to use for densities, point distances...etc.

All_Crimes = gdf.loc[gdf["Type"]=="Crimes",:]
All_Schools = gdf.loc[gdf["Type"]=="Schools",:]
Hospitals = gdf.loc[gdf["Type"]=="Hospital",:]
RailwayStations = gdf.loc[gdf["Type"]=="RailwayStations",:]
FerryTerminals = gdf.loc[gdf["Type"]=="FerryTerminals",:]
BusStops = gdf.loc[gdf["Type"]=="BusStops",:]
ListedBuildings = gdf.loc[gdf["Type"]=="Listed Buildings",:]
PropertySales = gdf.loc[gdf["Type"]=="LReg",:]
RoadAccidents = gdf.loc[gdf["Type"]=="RoadAccidents", :]


TotalRainfall_mm_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='TotalRainfall_mm_2020'),:]
Snow_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Snow_Days_2020'),:]
Sunlight_h_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Sunlight_h_2020'),:]
AverageTemperature_C_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='AverageTemperature_C_2020'),:]
GroundFrost_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='GroundFrost_Days_2020'),:]
#...etc.

#Point datasets to find the density of and the radius of those densities in the format output_name:dataframe where the output variable will have the name Outname_Density_Radius
density_var_dict = {
    "Postcodes":Postcodes, 
    "All_Crimes":All_Crimes, 
    "All_Schools":All_Schools, 
    "Hospitals":Hospitals, 
    "RailwayStations":RailwayStations,
    "FerryTerminals":FerryTerminals,
    "BusStops":BusStops,
    "ListedBuildings":ListedBuildings,
    "RoadAccidents":RoadAccidents
}

density_r_list = [1000,5000,25000]

#Point datasets to find the nearest of in the format output_name:dataframe where the output variable will have the name Nearest_Out_Name
nearest_var_dict = {
    "School":All_Schools, 
    "Hospital":Hospitals, 
    "RailwayStation":RailwayStations,
    "FerryTerminals":FerryTerminals,
    "BusStops":BusStops,
    "ListedBuilding":ListedBuildings
}

#Points to find the average of
average_var_dict = {
    "ProertyPrice_2020_2021":[PropertySales, "Details_Float"],
    "TotalRainfall_mm_2020":[TotalRainfall_mm_2020, "Details_Float"],
    "Snow_Days_2020":[Snow_Days_2020, "Details_Float"],
    "Sunlight_h_2020":[Sunlight_h_2020, "Details_Float"],
    "AverageTemperature_C_2020":[AverageTemperature_C_2020, "Details_Float"],
    "GroundFrost_Days_2020":[GroundFrost_Days_2020, "Details_Float"]
}

In [ ]:
#Start with adding density columns
for k in density_var_dict:
    for r in density_r_list:
        Postcodes[k + "_Density_" + str(r)] = kf.within_radius(Postcodes,density_var_dict[k], r)


In [ ]:
#Add distance to the nearest, don't include geometry lines
for k in nearest_var_dict:
    Postcodes[k + "_Distance_to_Nearest"] = kf.dist_to_nearest(Postcodes, nearest_var_dict[k], return_geom = False)

In [ ]:
#Find the average value within the list of radii
for k in average_var_dict:
    for r in density_r_list:
        Postcodes["Average_" + k + "_" + str(r)] = kf.average_within_radius(Postcodes, average_var_dict[k][0], average_var_dict[k][1], r)

In [ ]:
Postcodes.columns

Index(['geometry', 'Postcodes_Density_1000', 'Postcodes_Density_5000',
       'Postcodes_Density_25000', 'All_Crimes_Density_1000',
       'All_Crimes_Density_5000', 'All_Crimes_Density_25000',
       'All_Schools_Density_1000', 'All_Schools_Density_5000',
       'All_Schools_Density_25000', 'Hospitals_Density_1000',
       'Hospitals_Density_5000', 'Hospitals_Density_25000',
       'RailwayStations_Density_1000', 'RailwayStations_Density_5000',
       'RailwayStations_Density_25000', 'FerryTerminals_Density_1000',
       'FerryTerminals_Density_5000', 'FerryTerminals_Density_25000',
       'BusStops_Density_1000', 'BusStops_Density_5000',
       'BusStops_Density_25000', 'ListedBuildings_Density_1000',
       'ListedBuildings_Density_5000', 'ListedBuildings_Density_25000',
       'RoadAccidents_Density_1000', 'RoadAccidents_Density_5000',
       'RoadAccidents_Density_25000', 'Nearest_School_Distance',
       'Nearest_Hospital_Distance', 'Nearest_RailwayStation_Distance',
       'Near

In [ ]:
Postcodes.head()

,geometry,Postcodes_Density_1000,Postcodes_Density_5000,Postcodes_Density_25000,All_Crimes_Density_1000,All_Crimes_Density_5000,All_Crimes_Density_25000,All_Schools_Density_1000,All_Schools_Density_5000,All_Schools_Density_25000,...,Average_Snow_Days_2020_25000,Average_Sunlight_h_2020_1000,Average_Sunlight_h_2020_5000,Average_Sunlight_h_2020_25000,Average_AverageTemperature_C_2020_1000,Average_AverageTemperature_C_2020_5000,Average_AverageTemperature_C_2020_25000,Average_GroundFrost_Days_2020_1000,Average_GroundFrost_Days_2020_5000,Average_GroundFrost_Days_2020_25000
Postcode,,,,,,,,,,,,,,,,,,,,,
CM233WE,POINT (530754.000 182754.000),6038,33949,34071,21848,166699,168016,21,158,160,...,0.025311,1730.156347,1732.923776,1732.923776,12.787322,12.794034,12.794034,58.065582,55.770707,55.770707
E11AB,POINT (534253.000 181031.000),2171,33935,34071,14014,167159,168016,18,155,160,...,0.025311,1735.767290,1732.923776,1732.923776,12.793385,12.794034,12.794034,53.673690,55.770707,55.770707
E11AE,POINT (534279.000 181467.000),1968,33704,34071,16946,165690,168016,19,156,160,...,0.025311,1736.629681,1732.923776,1732.923776,12.769558,12.794034,12.794034,53.609399,55.770707,55.770707
E11AF,POINT (534253.000 181031.000),2171,33935,34071,14014,167159,168016,18,155,160,...,0.025311,1735.767290,1732.923776,1732.923776,12.793385,12.794034,12.794034,53.673690,55.770707,55.770707
E11AG,POINT (534291.000 181451.000),1915,33694,34071,16631,165690,168016,19,156,160,...,0.025311,1736.629681,1732.923776,1732.923776,12.769558,12.794034,12.794034,53.609399,55.770707,55.770707
